## Prelimiary Operations

### Manage the directories

In [1]:
# Not important
import os
os.listdir("../input/an2dlhw1bal-dataset/splitted_dataset")

['val', 'test', 'train']

In [2]:
# To delete a folder
import shutil
if os.path.exists("/kaggle/working/andrea"):
    shutil.rmtree("/kaggle/working/andrea")

In [3]:
# to remove a file
import os
if os.path.exists("/kaggle/working/model.png"):
    os.remove("/kaggle/working/model.png")

In [4]:
# to create a folder
import os
directory = "EfficientNet"
parent_dir = "/kaggle/working"
path = os.path.join(parent_dir, directory)
if not os.path.exists(path):
    os.mkdir(path)

### Operations

In [6]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from PIL import Image

tfk = tf.keras
tfkl = tf.keras.layers

2022-11-22 12:30:05.609374: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [7]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [8]:
import warnings
import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)

tf.get_logger().setLevel(logging.ERROR)
tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

## Importing data

In [9]:
img_height = 96
img_width =96
batch_size = 32

In [10]:
# Dataset folders. It is a single folder since we don't need a validation set (OOB validation)
dataset_dir = '../input/balanced-dataset-no-split/dataset'

In [11]:
# Create the dataset
dataset = tfk.preprocessing.image_dataset_from_directory(dataset_dir, label_mode='categorical', image_size=(img_height, img_width))

Found 4296 files belonging to 8 classes.


2022-11-22 12:30:11.348259: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-11-22 12:30:11.351917: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-11-22 12:30:11.417475: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-22 12:30:11.418173: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2022-11-22 12:30:11.418244: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-11-22 12:30:11.455514: I tensorflow/stream_executor/platform/def

In [12]:
# Create the X and y tensors from the dataset
X = None
y = None
for images, labels in dataset.as_numpy_iterator():
    # Create the tensor of the images in the batch
    t_img = tf.constant(images)
    if X is not None:   # if this is not the first batch
        # Concatenate the images and labels to the previous one
        X = tf.concat([X,t_img], axis=0)
        y = np.concatenate((y,labels), axis=0)
    else:   # if it is the first batch, simply copy the data into X and y
        X = t_img
        y = labels
print(X.shape)
print(y.shape)

2022-11-22 12:30:13.674974: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-11-22 12:30:13.678927: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2000175000 Hz


(4296, 96, 96, 3)
(4296, 8)


In [13]:
!cd /kaggle/working/EfficientNet

In [14]:
# Generator to perform data augmentation

def create_im_data_gen(X_train, y_train, X_test, y_test):
    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    # Create an instance of ImageDataGenerator, and for the trainign with Data Augmentation

    train_data_gen = ImageDataGenerator(rotation_range=20,
                                            height_shift_range=0.2,
                                            width_shift_range=0.2,
                                            zoom_range=0.2,
                                            horizontal_flip=True,
                                            shear_range=0.2,
                                            fill_mode='reflect',
                                            rescale=1/255.)

    valid_data_gen = ImageDataGenerator(rescale=1/255.)

    train_gen = train_data_gen.flow(X_train,
                                    y=y_train,
                                    batch_size=32,
                                    shuffle=True,
                                    seed=seed)

    valid_gen = valid_data_gen.flow(X_test,
                                    y=y_test,
                                    batch_size=32,
                                    shuffle=False,
                                    seed=seed)
    return train_gen, valid_gen

In [15]:
# Download and plot the EfficientNet model
eff_net = tfk.applications.efficientnet.EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(96,96,3)
)

16711680/16705208 [==============================] - 0s 0us/step


In [16]:
from tensorflow.python.framework.tensor_util import GetNumpyAppendFn

def build_model(input_shape, supernet, l1=None, l2=None, learning_rate=1e-4, model_name="efficient_net"):

    # Use the supernet as feature extractor
    supernet.trainable = True
    # in this way we keep the weights of the CNN part and we will train only the weights of the classifier

    input_layer = tfkl.Input(shape=input_shape, name='input_layer')
    x = supernet(input_layer)
    x = tfkl.GlobalAveragePooling2D()(x)
    x = tfkl.Dropout(0.3, seed=seed)(x)
    outputs = tfkl.Dense(
        8, 
        activation='softmax',
        kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)


    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=outputs, name=model_name)
    
    model.get_layer('efficientnetb0').trainable = True
    
    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.RMSprop(learning_rate), metrics='accuracy')
    
    
    return model

In [17]:
# Main function that train and evaluate a single model, given the bootstrapped dataset

def evaluate_model(X_train, y_train, X_test, y_test, model_name="efficient_net"):
    train_gen, valid_gen = create_im_data_gen(X_train, y_train, X_test, y_test)
    
    input_shape = (96, 96, 3)
    epochs = 300
    l1=1e-7
    l2=1e-4
    learning_rate = 1e-4
    patience = 20
    
    model = build_model(input_shape, eff_net, l1=l1, l2=l2, learning_rate=learning_rate, model_name=model_name)
    
    # Train the model
    model_history = model.fit(
        x = train_gen,
        batch_size = 256,
        epochs = epochs,
        validation_data = valid_gen,
        callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=patience, restore_best_weights=True)],
    ).history
    
    # Predict the test set with the CNN
    predictions = model.predict(X_test)

    # Compute the confusion matrix
    cm = confusion_matrix(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1))

    # Compute the classification metrics
    accuracy = accuracy_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1))
    precision = precision_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro')
    recall = recall_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro')
    f1 = f1_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro')
    print('Accuracy:',accuracy.round(4))
    print('Precision:',precision.round(4))
    print('Recall:',recall.round(4))
    print('F1:',f1.round(4))
    score = {'confusion_matrix': cm, 'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1}
    
    return model, score
    
    

In [18]:
from sklearn.utils import resample

# multiple train-test splits
n_splits = 10
scores = list()
n_samples = len(X)
for i in range(n_splits):
    # select indexes
    ix = [i for i in range(n_samples)]
    train_ix = resample(ix, replace=True, n_samples=n_samples)
    test_ix = [x for x in ix if x not in train_ix]
    # select data
    X_train, y_train = tf.gather(X, indices=train_ix), y[train_ix]
    X_test, y_test = tf.gather(X, indices=test_ix), y[test_ix]
    # evaluate model
    model_name = "efficient_net" + str(i)
    model, score = evaluate_model(X_train, y_train, X_test, y_test, model_name=model_name)
    
    # Save all the models in different directories, to be used after for prediction
    directory = "EfficientNet" + str(i)
    parent_dir = "/kaggle/working/EfficientNet"
    path = os.path.join(parent_dir, directory)
    if os.path.exists(path):
        shutil.rmtree(path)
    os.mkdir(path)
    model.save(path)
    del model
    
    scores.append(score)

Epoch 1/300


2022-11-22 12:30:47.859601: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-11-22 12:30:48.773976: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2022-11-22 12:30:48.829742: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8


135/135 [==============================] - 36s 128ms/step - loss: 1.9673 - accuracy: 0.2440 - val_loss: 2.1480 - val_accuracy: 0.1329
Epoch 2/300
135/135 [==============================] - 15s 110ms/step - loss: 1.3709 - accuracy: 0.4951 - val_loss: 2.1376 - val_accuracy: 0.1237
Epoch 3/300
135/135 [==============================] - 15s 114ms/step - loss: 1.1245 - accuracy: 0.5910 - val_loss: 2.3583 - val_accuracy: 0.1268
Epoch 4/300
135/135 [==============================] - 15s 109ms/step - loss: 0.9716 - accuracy: 0.6480 - val_loss: 3.4999 - val_accuracy: 0.1133
Epoch 5/300
135/135 [==============================] - 16s 114ms/step - loss: 0.8334 - accuracy: 0.6909 - val_loss: 3.0788 - val_accuracy: 0.1457
Epoch 6/300
135/135 [==============================] - 15s 109ms/step - loss: 0.7496 - accuracy: 0.7244 - val_loss: 2.6640 - val_accuracy: 0.1960
Epoch 7/300
135/135 [==============================] - 16s 116ms/step - loss: 0.6611 - accuracy: 0.7628 - val_loss: 2.2444 - val_accurac

2022-11-22 12:50:04.947963: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Epoch 1/300
135/135 [==============================] - 32s 121ms/step - loss: 0.2269 - accuracy: 0.9336 - val_loss: 2.8072 - val_accuracy: 0.2232
Epoch 2/300
135/135 [==============================] - 18s 131ms/step - loss: 0.1592 - accuracy: 0.9443 - val_loss: 2.1065 - val_accuracy: 0.4021
Epoch 3/300
135/135 [==============================] - 15s 113ms/step - loss: 0.1242 - accuracy: 0.9606 - val_loss: 2.2618 - val_accuracy: 0.4084
Epoch 4/300
135/135 [==============================] - 17s 126ms/step - loss: 0.1174 - accuracy: 0.9588 - val_loss: 5.1102 - val_accuracy: 0.1777
Epoch 5/300
135/135 [==============================] - 15s 114ms/step - loss: 0.1241 - accuracy: 0.9651 - val_loss: 2.0130 - val_accuracy: 0.5673
Epoch 6/300
135/135 [==============================] - 17s 127ms/step - loss: 0.0932 - accuracy: 0.9684 - val_loss: 2.9508 - val_accuracy: 0.2244
Epoch 7/300
135/135 [==============================] - 15s 114ms/step - loss: 0.1043 - accuracy: 0.9693 - val_loss: 4.1800 -

In [19]:
# Print the validation accuracy of each model on their evaluation set
for i in range(n_splits):
    print("Accuracy of ", i+1, " NN: ", scores[i]['accuracy'])
    
    directory = "EfficientNet" + str(i)
    parent_dir = "/kaggle/working/EfficientNet"
    path = os.path.join(parent_dir, directory)
    np.savez_compressed(path, scores=scores[i])

Accuracy of  1  NN:  0.1132884262094305
Accuracy of  2  NN:  0.12718204488778054
Accuracy of  3  NN:  0.11132315521628498
Accuracy of  4  NN:  0.11577608142493638
Accuracy of  5  NN:  0.14140773620798985
Accuracy of  6  NN:  0.130625
Accuracy of  7  NN:  0.12795905310300704
Accuracy of  8  NN:  0.11962025316455696
Accuracy of  9  NN:  0.10236724248240563
Accuracy of  10  NN:  0.11494252873563218


In [20]:
# Create the zip file
import shutil
shutil.make_archive("Efficient_net_bagging", 'zip', "/kaggle/working/EfficientNet")

'/kaggle/working/Efficient_net_bagging.zip'